In [51]:
!pip3 install openai
!pip3 install asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
import pandas

questions = pandas.read_csv('https://raw.githubusercontent.com/bamartin1618/1WeekDataset/main/Question_Classification_Dataset.csv')
question_sample = questions[questions.columns[1]].sample(50).to_numpy()

# Approach 1: Single Requests
<p>The following code answers fifty questions using the GPT-3 API, sending one request at a time.</p>
<p>Straightforward but slow.</p>

In [53]:
import os
import openai
import time
import aiohttp
import asyncio

openai.api_key = str(os.getenv('GPT_KEY'))

t0 = time.time()

answers = []

for question in question_sample:

  response = openai.Completion.create(
      model='text-ada-001',
      prompt=question,
      temperature=0.7,
      max_tokens=100,
      top_p=1
  )

  answers.append(response['choices'][0]['text'])

t1 = time.time()
print(t1-t0)
print(answers)
print(len(answers))

13.091099262237549
['\n\nThe year is 2020.', '\n\nJohn C. Calhoun and Henry Clay were two of the most important and influential Members of the House of Representatives of the United States Congress.', '\n\nThere are only 10 students at the University of Massachusetts.', '\n\nThe most common surname in America is "Jones".', '\n\nROI: Returns on Investment for CCD technology', '\n\nA fear of motion is a fear of being able to move. This fear can come from a fear of being in a moving car, of being able to see in a dark place, or of moving too much.', '\n\nThe legal process typically includes a case conference, which is a meeting with a lawyer to discuss the best way to go about fighting a case.', '\n\nThere are a few steps you need to take in order to start writing for television. First, you need to develop a writing style and find a topic that interests you. Next, you need to find people who are interested in writing about your topic and try to produce a letter of proposal before the latt

# Approach 2: Concurrent Requests
<p>It's really easy to make concurrent requests with the GPT-3 API, and there's a huge boost in speed.</p>
<p>You can do it by taking the endpoint that the GPT Playground sends requests to through the Network Tab in Developer Tools. The rest is simple.</p>

In [54]:
!pip3 install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
from pickle import TUPLE2
import nest_asyncio
nest_asyncio.apply()

async def get_text(session, url, params):

    async with session.post(url, json = params) as resp:
        text = await resp.json()
        return text['choices'][0]['text']

async def answer_questions():

  async with aiohttp.ClientSession(headers = {'authorization' : f"Bearer {os.getenv('GPT_TOKEN')}"}) as session:

    jobs = []
    
    for question in question_sample:
      url = 'https://api.openai.com/v1/engines/text-ada-001/completions'
      jobs.append(asyncio.ensure_future(get_text(session, url, {'prompt' : 'question', 'temperature' : 0.7, 'max_tokens' : 100, 'top_p' : 1})))

    responses = await asyncio.gather(*jobs)

  return responses

t2 = time.time()
answers = asyncio.run(answer_questions())
t3 = time.time()

print(t1-t0)
print(len(answers))

13.091099262237549
50


In [56]:
print(f'Single Requests (Time to Complete): {t1-t0} seconds')
print(f'Concurrent Requests (Time to Complete): {t3-t2} seconds')

Single Requests (Time to Complete): 13.091099262237549 seconds
Concurrent Requests (Time to Complete): 0.8138439655303955 seconds
